# Minimal crowd simulation example

This guide explains how to run multiple simulations using the **CrowdMechanics** library and how to manage their outputs.

---

## 1. Export crowd configuration files

1. Open the [**Streamlit app**](https://crowdmecha.streamlit.app/).
2. Go to the **Crowd** tab.
3. Click **"Initialize your own crowd"**.
4. In the sidebar at the bottom, click **"Export crowd as XML config files"**.
5. Download the XML files and save them in an appropriate location.

---

## 2. Set the working environnement

Create two folders in your project directory `static` and `dynamic`. Place the following files that you downloaded from the Streamlit app in the respective folders:
  - Move `Agents.xml`, `Geometry.xml`, and `Materials.xml` into the `static` folder.
  - Move `AgentDynamics.xml` into the `dynamic` folder.

Create a `Parameters.xml` file in your actual folder location. Use the following template (replace the paths with your actual folder locations):

```xml
<?xml version="1.0" encoding="utf-8"?>
<Parameters>
    <Directories Static="./static/" Dynamic="./dynamic/"/>
    <Times TimeStep="0.1" TimeStepMechanical="5e-6"/>
</Parameters>
```

**Directory structure example:**
```
.
├── Parameters.xml
├── static/
|   ├── Agents.xml
│   ├── Geometry.xml
│   └── Materials.xml
└── dynamic/
    └── AgentDynamics.xml
```

---

## 3. Build the C++ project

Navigate to the root of the `mechanical_layer` directory and build the project:

```bash
cmake -H. -Bbuild -DBUILD_SHARED_LIBS=ON
cmake --build build
```

---

## 4. Run a series of simulations

Execute the Python code below, modifying it as needed for your experiments.
Simulation results are automatically saved in the `outputXML/` directory.
Each output file is named as follows:
```
AgentDynamics output t=TIME_VALUE.xml
```
Here, `TIME_VALUE` represents the specific simulation time or identifier for that run.



In [1]:
import ctypes
from pathlib import Path
import numpy as np
from shutil import copyfile
import xml.etree.ElementTree as ET

### Parameters
dt = 0.1  # The "TimeStep" in Parameters.xml
Ndt = 100  # The number of successive calls to the library

outputPath = "outputXML/"
inputPath = "inputXML/"
Path(outputPath).mkdir(parents=True, exist_ok=True)
Path(inputPath).mkdir(parents=True, exist_ok=True)

# Load the library into ctypes
Clibrary = ctypes.CDLL("../../src/mechanical_layer/build/libCrowdMechanics.dylib")  # Use .so for Linux and .dylib for MacOS
# The file name for the dynamical quantities will be used to build the names of the output files
agentDynamicsFilename = "AgentDynamics.xml"
# Prepare the call to CrowdMechanics
files = [b"Parameters.xml", b"Materials.xml", b"Geometry.xml", b"Agents.xml", agentDynamicsFilename.encode("ascii")]
nFiles = len(files)
filesInput = (ctypes.c_char_p * nFiles)()
filesInput[:] = files


### Actual loop
for t in range(Ndt):
    print("Looping the Crowd mechanics engine - t=%.1fs..." % (t * dt))
    # Copy Agent dynamics input file
    copyfile("dynamic/" + agentDynamicsFilename, inputPath + rf"AgentDynamics input t={t * dt:.1f}.xml")
    # Call the mechanical layer
    Clibrary.CrowdMechanics(filesInput)
    # Copy Agent dynamics output file to the directory that will be read by ChAOS
    copyfile("dynamic/" + agentDynamicsFilename, outputPath + rf"AgentDynamics output t={(t + 1) * dt:.1f}.xml")
    # Save the AgentInteractions file, if it exists
    try:
        copyfile("dynamic/AgentInteractions.xml", outputPath + rf"AgentInteractions t={(t + 1) * dt:.1f}.xml")
    except FileNotFoundError:
        pass
    # Prepare next run: Add dynamics tag to the input file
    # This is the step where you decide the Fp and Mp that will drive the next dt seconds for each agent
    XMLtree = ET.parse("dynamic/" + agentDynamicsFilename)
    agentsTree = XMLtree.getroot()

    ######################################################################################
    ####### Sample dummy code where we put the same constant values for each agent #######
    ####### (can be replaced with your own decisional layer code)                  #######
    for agent in agentsTree:
        dynamicsItem = ET.SubElement(agent, "Dynamics")
        # set a random force and random moment but with a mean value of 70N and 0Nm for FP
        random_force_x = np.random.normal(loc=200, scale=200)
        random_force_y = np.random.normal(loc=0, scale=50)
        random_moment = np.random.normal(loc=0, scale=5)
        dynamicsItem.attrib["Fp"] = f"{random_force_x:.2f},{random_force_y:.2f}"
        dynamicsItem.attrib["Mp"] = f"{random_moment:.2f}"
    ######################################################################################

    XMLtree.write("dynamic/" + agentDynamicsFilename)

# Done!
print(f"Loop terminated at t={Ndt * dt:.1f}s!")

Looping the Crowd mechanics engine - t=0.0s...
Looping the Crowd mechanics engine - t=0.1s...
Looping the Crowd mechanics engine - t=0.2s...
Looping the Crowd mechanics engine - t=0.3s...
Looping the Crowd mechanics engine - t=0.4s...
Looping the Crowd mechanics engine - t=0.5s...
Looping the Crowd mechanics engine - t=0.6s...
Looping the Crowd mechanics engine - t=0.7s...
Looping the Crowd mechanics engine - t=0.8s...
Looping the Crowd mechanics engine - t=0.9s...
Looping the Crowd mechanics engine - t=1.0s...
Looping the Crowd mechanics engine - t=1.1s...
Looping the Crowd mechanics engine - t=1.2s...
Looping the Crowd mechanics engine - t=1.3s...
Looping the Crowd mechanics engine - t=1.4s...
Looping the Crowd mechanics engine - t=1.5s...
Looping the Crowd mechanics engine - t=1.6s...
Looping the Crowd mechanics engine - t=1.7s...
Looping the Crowd mechanics engine - t=1.8s...
Looping the Crowd mechanics engine - t=1.9s...
Looping the Crowd mechanics engine - t=2.0s...
Looping the C

For a c++ example file and precise explanations, read the `Mechanical layer` tutorial.

---

## 5. Export to ChAOS

The output files can be easily converted into `.csv` files structured for direct import into [ChAOS](https://project.inria.fr/crowdscience/project/ocsr/chaos/). ChAOS uses these CSV files—containing time, x, and y coordinates for each agent—to visualize and animate agent trajectories for further analysis and video generation.

In [2]:
from configuration.backup import xml_to_Chaos

filenameCSV = "all_trajectories.csv"
PathXML = Path("outputXML").resolve()
PathCSV = Path("outputCSV").resolve()
PathCSV.mkdir(parents=True, exist_ok=True)

xml_to_Chaos.export_dict_to_CSV(PathCSV, PathXML)
xml_to_Chaos.export_from_CSV_to_CHAOS(PathCSV, dt)

Processing file: /Users/oscardufour/Documents/LEMONS/tutorials/mechanical_layer/outputXML/AgentDynamics output t=5.9.xml
Processing file: /Users/oscardufour/Documents/LEMONS/tutorials/mechanical_layer/outputXML/AgentDynamics output t=9.4.xml
Processing file: /Users/oscardufour/Documents/LEMONS/tutorials/mechanical_layer/outputXML/AgentDynamics output t=9.5.xml
Processing file: /Users/oscardufour/Documents/LEMONS/tutorials/mechanical_layer/outputXML/AgentDynamics output t=5.8.xml
Processing file: /Users/oscardufour/Documents/LEMONS/tutorials/mechanical_layer/outputXML/AgentDynamics output t=7.8.xml
Processing file: /Users/oscardufour/Documents/LEMONS/tutorials/mechanical_layer/outputXML/AgentDynamics output t=9.7.xml
Processing file: /Users/oscardufour/Documents/LEMONS/tutorials/mechanical_layer/outputXML/AgentDynamics output t=9.6.xml
Processing file: /Users/oscardufour/Documents/LEMONS/tutorials/mechanical_layer/outputXML/AgentDynamics output t=7.9.xml
Processing file: /Users/oscarduf

---

## 6. Export to a movie with FFMPEG

A plot of the scene can be generated from each output file under `PNG` format. 

In [3]:
import os
import matplotlib.pyplot as plt
import configuration.backup.dict_to_xml_and_reverse as fun_xml
from configuration.models.crowd import create_agents_from_dynamic_static_geometry_parameters
from streamlit_app.plot import plot


# --- Prepare the folders ---
staticPath = Path("./static")
moviesPath = Path("./movies")
plotsPath = Path("./plots")
plotsPath.mkdir(parents=True, exist_ok=True)
moviesPath.mkdir(parents=True, exist_ok=True)
for file in plotsPath.glob("*.png"):
    os.remove(file)

# --- Load static XML files ---
with open(staticPath / "Agents.xml", encoding="utf-8") as f:
    crowd_xml = f.read()
static_dict = fun_xml.static_xml_to_dict(crowd_xml)

with open(staticPath / "Geometry.xml", encoding="utf-8") as f:
    geometry_xml = f.read()
geometry_dict = fun_xml.geometry_xml_to_dict(geometry_xml)

# --- Loop over time steps ---
for t in range(Ndt):
    current_time = (t + 1) * dt
    filename = f"AgentDynamics output t={current_time:.1f}.xml"
    dynamics_file = Path(outputPath) / filename

    if not dynamics_file.exists():
        print(f"Warning: {dynamics_file} not found, skipping.")
        continue

    # Read the dynamics XML file for the current time step
    with open(dynamics_file, encoding="utf-8") as f:
        dynamic_xml = f.read()
    dynamic_dict = fun_xml.dynamic_xml_to_dict(dynamic_xml)

    # Create the Crowd object and populate it with the data from the dictionaries
    crowd = create_agents_from_dynamic_static_geometry_parameters(
        static_dict=static_dict,
        dynamic_dict=dynamic_dict,
        geometry_dict=geometry_dict,
    )

    plot.display_crowd2D(crowd)
    plt.savefig(plotsPath / rf"crowd2D_t={t:d}.png", dpi=300, format="png")
    plt.close()

All the `PNG` images can then be combined into a video using [FFmpeg](https://ffmpeg.org/).

In [4]:
import subprocess

ffmpeg = "/Users/oscardufour/ffmpeg_bin/ffmpeg"
movie_name = "example"
framerate = int(1.0 / dt)

bashCommand1 = f"{ffmpeg}  -framerate {framerate} -i {str(plotsPath)}/crowd2D_t=%d.png {str(moviesPath)}/{movie_name}.mp4"
process = subprocess.Popen(bashCommand1.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

bashCommand1 = f"{ffmpeg}  -i {str(moviesPath)}/{movie_name}.mp4 -pix_fmt yuv420p {str(moviesPath)}/{movie_name}.mov"
process = subprocess.Popen(bashCommand1.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

bashCommand1 = f"rm {str(moviesPath)}/{movie_name}.mp4"
process = subprocess.Popen(bashCommand1.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

ffmpeg version 7.0.2-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libharfbuzz --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxml2 --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --disable-videotoolbox -